In [1]:
import numpy as np

In [2]:
KNAPSACK_CAPACITY = 50 # Maximum weight the knapsack can hold
POPULATION_SIZE = 20 # Number of individuals in the population
MAX_NO_OF_GENERATIONS = 20 # Maximum number of generations
NO_OF_ITEMS = 15 # Number of items to be included in the knapsack
MAX_ITEM_WEIGHT = 20 # Maximum weight of an item

np.random.seed(0)

# Generate random items 
ITEMS = [
    {
        "value": np.random.randint(0, MAX_ITEM_WEIGHT), 
        "weight": np.random.randint(0, MAX_ITEM_WEIGHT)
    } for _ in range(NO_OF_ITEMS)
]
ITEMS

[{'value': 12, 'weight': 15},
 {'value': 0, 'weight': 3},
 {'value': 3, 'weight': 7},
 {'value': 9, 'weight': 19},
 {'value': 18, 'weight': 4},
 {'value': 6, 'weight': 12},
 {'value': 1, 'weight': 6},
 {'value': 7, 'weight': 14},
 {'value': 17, 'weight': 5},
 {'value': 13, 'weight': 8},
 {'value': 9, 'weight': 19},
 {'value': 16, 'weight': 19},
 {'value': 5, 'weight': 15},
 {'value': 15, 'weight': 0},
 {'value': 18, 'weight': 3}]

In [3]:
def generate_population(
    # Number of individuals in the population
    population_size: int,
    # Number of items to be included in the knapsack
    no_of_items: int 
) -> list[list[int]]:
    
    population = []
    for _ in range(population_size):
        chromosome = list(np.random.randint(0, 2, no_of_items))
        population.append(chromosome)

    return population

# Generate the initial population
POPULATION = generate_population(POPULATION_SIZE, NO_OF_ITEMS)
POPULATION

[[0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1],
 [0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
 [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1],
 [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1],
 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1],
 [0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1],
 [1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1],
 [0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1],
 [0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0],
 [0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]]

In [4]:
def fitness(
    # The chromosome to be evaluated
    chromosome: list[int], 
    # The items to be included in the knapsack
    items: list[dict[str, int]], 
    # The maximum weight the knapsack can hold
    knapsack_capacity: int, 
    # The penalty factor for exceeding the knapsack capacity
    penalty_factor: int = 6 
) -> int:
    
    weight = 0
    value = 0
    for i in range(min(len(chromosome), len(items))):
        if chromosome[i] == 1:
            weight += items[i]["weight"]
            value += items[i]["value"]

    penalty = 0
    if weight > knapsack_capacity:
        penalty = penalty_factor*(weight - knapsack_capacity)

    return value - penalty
    
def evolve_population(
    # The population of chromosomes
    population: list[list[int]], 
    # The percentage of the population that will be taken over without changes
    parent_percent: float = 0.2, 
    # The probability of a gene being mutated
    mutation_chance: float = 0.08 
) -> list[list[int]]:
    
    # Select parents
    parents = population[:int(parent_percent * len(population))]

    # Perform single point crossover
    children = []
    while len(children) < len(population) - len(parents):
        father = parents[np.random.randint(0, len(parents))]
        mother = parents[np.random.randint(0, len(parents))]
        crossover_point = int(np.random.randint(0, len(father)))
        child = father[:crossover_point] + mother[crossover_point:]
        children.append(child)

    # Perform mutation
    for child in children:
        if np.random.rand() < mutation_chance:
            mutation_point = np.random.randint(0, len(child))
            child[mutation_point] = 1 - child[mutation_point]

    # Combine parents and children to form the new population
    parents.extend(children)
    return parents

# Evolve the population
for generation_id in range(1, MAX_NO_OF_GENERATIONS+1):
    POPULATION.sort(key=lambda x: fitness(x, ITEMS, KNAPSACK_CAPACITY), reverse=True)
    total_fitness = sum(fitness(chromosome, ITEMS, KNAPSACK_CAPACITY) for chromosome in POPULATION)
    print(f"Generation {generation_id:3d}, total fitness: {total_fitness:4d}", end=", ")

    POPULATION = evolve_population(POPULATION)
    POPULATION.sort(key=lambda x: fitness(x, ITEMS, KNAPSACK_CAPACITY), reverse=True)
    print(f"best solution: {''.join(map(lambda x: str(x), POPULATION[0]))}", end=", ")
    print(f"fitness: {fitness(POPULATION[0], ITEMS, KNAPSACK_CAPACITY):3d}")

Generation   1, total fitness: -1678, best solution: 010010111100011, fitness:  89
Generation   2, total fitness: 1195, best solution: 010010111100011, fitness:  89
Generation   3, total fitness: 1780, best solution: 010010111100011, fitness:  89
Generation   4, total fitness: 1779, best solution: 010010111100011, fitness:  89
Generation   5, total fitness: 1780, best solution: 010010111100011, fitness:  89
Generation   6, total fitness: 1717, best solution: 010010111100011, fitness:  89
Generation   7, total fitness: 1735, best solution: 010010111100011, fitness:  89
Generation   8, total fitness: 1780, best solution: 010010111100011, fitness:  89
Generation   9, total fitness: 1767, best solution: 011010111100011, fitness:  92
Generation  10, total fitness: 1747, best solution: 011010111100011, fitness:  92
Generation  11, total fitness: 1781, best solution: 011010111100011, fitness:  92
Generation  12, total fitness: 1840, best solution: 011010111100011, fitness:  92
Generation  13,